In [1]:
import json
from gcms_spectra_gnn.spectra_dataset import MoleculesDataset, Preprocessor
import numpy as np
import os

In [2]:
mgf_json_file = 'data/NIST_GC_EI_MAX_LIBRARY.json'

In [3]:
with open(mgf_json_file) as fh:
    library = json.load(fh)

In [13]:
len(library)

193121

In [5]:
class FileNameGetter:
    
    MAX_PER_DIR = 10000
    
    def __init__(self, prefix, lib_path):
        self.lib_path = lib_path
        self.prefix = prefix
        
    def get_filename(self, entry):
        n = entry['SCANS']
        base_path = os.path.join(self.prefix, str(int(n) // self.MAX_PER_DIR + 1))
        full_path = os.path.join(self.lib_path, base_path)
        if not os.path.exists(full_path):
            os.makedirs(full_path)
        return os.path.join(base_path, n + '.npz')
        

In [27]:
prefix = ''
lib_path = 'data/preprocessed_full'
if not os.path.exists(lib_path):
    os.makedirs(lib_path)
BEGIN_IDX = 0
# END_IDX = 10000
END_IDX = len(library)
def slicer(x):
    return x[BEGIN_IDX:END_IDX]


filegen = FileNameGetter(prefix, lib_path)
spectra = [[entry['M/Z'], entry['INTENSITY']] for entry in slicer(library) if 'SMILES' in entry]
smiles = [entry['SMILES'] for entry in slicer(library) if 'SMILES' in entry]
filenames = [os.path.join(lib_path, filegen.get_filename(entry)) for entry in slicer(library) if 'SMILES' in entry]

preproc = Preprocessor(add_h=True)

preproc.transform(spectra, smiles, filenames)

saved_library = []
for saved, entry in zip(preproc.saved, slicer(library)):
    if saved:
        fp = filegen.get_filename(entry)  
        entry['FP_PATH'] = fp
        saved_library.append(entry)

with open(os.path.join(lib_path, 'index.json'), 'w') as fh:
    json.dump(saved_library, fh)

KeyboardInterrupt: 